In [1]:
## install open3d
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Unins

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, cos, sin, atan2
import copy
import pickle
import gzip
import pathlib
import os
import json

def load_poses(file_path):
    poses = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                x, y, z, roll, pitch, yaw = map(float, line.strip().split())
                poses.append((x, y, z, roll, pitch, yaw))
    return poses

def load_point_cloud_from_objects_pkl_gz(file_path, ids):
    """
    從有 data['objects'] 的 .pkl.gz 讀取並合併指定 IDs 的點雲。
    假設每個 object 都含 pcd_np (N,3) & pcd_color_np (N,3)。
    如果合併後的點雲數量大於 8192，則進行降採樣，並儲存為 (8192, 6) 格式。

    Parameters:
        file_path (str): .pkl.gz 檔案路徑。
        ids (list): 要合併的 object ID 列表。

    Returns:
        None: 結果會儲存為 .npy 檔案。
    """
    with gzip.open(file_path, "rb") as f:
        data = pickle.load(f)

    objects = data["objects"]  # list of dict
    if len(objects) > 0:
      print(f"[INFO] Sample object keys: {list(objects[0].keys())}")
      for key, value in objects[0].items():
          print(f"Key: {key}, Type: {type(value)}")
          if hasattr(value, "shape"):
              print(f"  Shape: {value.shape}")
    print(f"[INFO] Total objects: {len(objects)}")

    all_points = []
    all_colors = []
    # print(objects[0]["class_id"])

    for idx, obj in enumerate(objects, start=1):
        if idx in ids:  # 篩選符合的 ID
          id = obj["captions"]
          # print(id)
          pts = obj["pcd_np"]         # shape (N,3)
          cols = obj["pcd_color_np"]  # shape (N,3)
          if pts.shape[0] > 0:
              all_points.append(pts)
              all_colors.append(cols)

    if not all_points:
        print("[WARNING] No points found for the specified IDs.")
        return

    # 合併所有指定 IDs 的點雲
    points_np = np.concatenate(all_points, axis=0)  # shape (M, 3)
    colors_np = np.concatenate(all_colors, axis=0)  # shape (M, 3)

    # 合併點雲 (M, 6)
    combined_np = np.hstack((points_np, colors_np))  # shape (M, 6)

    # 如果點雲數量大於 8192，進行隨機降採樣
    num_points = combined_np.shape[0]
    if num_points > 8192:
        indices = np.random.choice(num_points, 8192, replace=False)
        combined_np = combined_np[indices]
    elif num_points < 8192:
        # 如果少於 8192，補齊至 8192
        additional_indices = np.random.choice(num_points, 8192 - num_points, replace=True)
        combined_np = np.vstack((combined_np, combined_np[additional_indices]))

    # 確保輸出形狀正確
    assert combined_np.shape == (8192, 6), "Output point cloud is not in shape (8192, 6)."

    # 儲存點雲
    output_filename = f"/content/drive/MyDrive/CG-PCD/point_cloud_ids_{len(ids)}.npy"
    np.save(output_filename, combined_np)
    print(f"[INFO] Saved combined point cloud to {output_filename}")
def generateObjectIDs(json_path, ids):
    """
    將 ids 中的值對應到 JSON 檔案中的 id，並回傳該 id 所代表的 Object 的 objID 列表。

    Parameters:
        json_path (str): JSON 檔案路徑。
        ids (list[int]): 要篩選的 ID 列表。

    Returns:
        list[int]: 回傳對應到的 objID 列表。
    """
    # 讀取 JSON 檔案
    with open(json_path, "r") as f:
        data = json.load(f)

    objName = []
    objID = []

    # 遍歷 JSON 中的物件
    for key, obj in data.items():
        if obj["id"] in ids:  # 檢查是否在目標 ID 中
            objName.append(key)
    for name in objName:
      objID.append(extract_id_from_key(name))
    return objID
def extract_id_from_key(key):
    """
    從鍵名中提取最後的數字或 `_` 後的數字，並轉換為整數。

    Parameters:
        key (str): 鍵名，例如 "object_1", "object_123"。

    Returns:
        int: 提取到的數字部分。
    """
    if "_" in key:
        return int(key.split("_")[-1])  # 分割後取最後一部分並轉為整數
    else:
        raise ValueError(f"Key '{key}' does not contain an underscore '_'.")

def main():
    pcd_file  = "/content/drive/MyDrive/CG-PCD/pcd_r_mapping_stride10.pkl.gz"
    pose_file = "/content/drive/MyDrive/CG-PCD/pose.txt"
    json_file = "/content/drive/MyDrive/CG-PCD/obj_json_r_mapping_stride10.json"

    ids = [10, 12, 19, 92, 93, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 134, 135, 136, 138, 141, 156]
    # objID = generateObjectIDs(json_file, ids)
    pcd = load_point_cloud_from_objects_pkl_gz(pcd_file, ids)
    poses = load_poses(pose_file)
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'open3d'